In [1]:
import os

In [2]:
%pwd

'd:\\Kumar\\Datascience_projects\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Kumar\\Datascience_projects\\Chicken-Disease-Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [12]:
pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install joblib

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

ImportError: cannot import name 'create_directories' from 'src.cnnClassifier.utils.common' (d:\Kumar\Datascience_projects\Chicken-Disease-Classification\src\cnnClassifier\utils\common.py)